# Qiskit Benchmarks

## Imports and setup

In [1]:
import qiskit
import mitiq
import time
import numpy as np
import collections

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.ibm_backend import IBMBackend

## Utils

In [2]:
backend_name = "ibm_kyiv"
backend = QiskitRuntimeService(
    channel="ibm_quantum",
    token="c0ed0f977fc96d511605341385f656a6f645fad36841935e8607da73c66b88d94fc4f0039f31fcb5deb38050ed690e95d6bdef43a2601351aff0d547a0da5af8"
).backend(backend_name)

In [32]:
class QiskitPassProfiler:
    def __init__(self):
        self.pass_timings = collections.defaultdict(list)
        
    def note_pass_timing(self, pass_, dag, time, property_set, count) -> None:
        self.pass_timings[pass_.name()].append(time)
    
profiler = QiskitPassProfiler()
staged_pass_manager = qiskit.transpiler.preset_passmanagers.generate_preset_pass_manager(3, backend, layout_method="sabre", routing_method="sabre")
# stages = ["init", "layout", "routing", "translation", "optimization", "scheduling"]

circuits = get_cdr_circuits()

# for stage in stages:
#     pre_stage_pass_manager = getattr(staged_pass_manager, f"pre_{stage}")
#     stage_pass_manager = getattr(staged_pass_manager, stage)
#     post_stage_pass_manager = getattr(staged_pass_manager, f"post_{stage}")
    
#     for pass_manager in [pre_stage_pass_manager, stage_pass_manager, post_stage_pass_manager]:
#         if pass_manager is not None:
#             print(pass_manager._tasks)

compiled_circuits = staged_pass_manager.run(circuits, callback=profiler.note_pass_timing, num_processes=1)

In [34]:
pass_latencies = {key: sum(value) for key, value in profiler.pass_timings.items()}

print(sum([item for item in pass_latencies.values()]))

7.782702445983887


In [100]:
def get_cnot_layer_indices(
    connectivity: dict[list[int]],
    qubits: list[int],
    max_cnots: int,
) -> np.ndarray:
    selected_cnot_indices = []
    visited = set()
    
    qubits_shuffled = np.random.permutation(qubits)
    qubits_set = set(qubits)
    
    for qubit in qubits_shuffled:
        if qubit in visited:
            continue
            
        for adj_qubit in connectivity[qubit]:
            if (adj_qubit not in visited) and (adj_qubit in qubits_set):
                visited.add(adj_qubit)
                selected_cnot_indices.append([qubit, adj_qubit])
                break
                
        visited.add(qubit)
        
        if len(selected_cnot_indices) == max_cnots:
            break         

    return np.array(selected_cnot_indices).reshape(-1, 2)


def add_local_pauli_measurements(
    circuit: qiskit.QuantumCircuit,
    num_qubits: int,
) -> qiskit.QuantumCircuit:
    pauli_measurement_basis_shifts = [
        [qiskit.circuit.library.IGate(), qiskit.circuit.library.IGate()],
        [qiskit.circuit.library.HGate(), qiskit.circuit.library.IGate()],
        [qiskit.circuit.library.SdgGate(), qiskit.circuit.library.HGate()],
    ]
    
    chosen_pauli_measurement_basis = np.random.randint(0, 3)
    
    for qubit in range(num_qubits):
        circuit.append(pauli_measurement_basis_shifts[chosen_pauli_measurement_basis][0], [qubit])
        circuit.append(pauli_measurement_basis_shifts[chosen_pauli_measurement_basis][1], [qubit])
        
    circuit.measure_all()
    
    return circuit

## Benchmark circuits

### Clifford Data Regression

In [34]:
def get_cdr_circuits() -> list[qiskit.QuantumCircuit]:
    num_qubits = 16
    num_clifford_circuits = 100
    circuits = []

    for _ in range(num_clifford_circuits):
        circuit = qiskit.QuantumCircuit(num_qubits)

        for i in range(num_qubits):
            circuit.h(i)
    
        for layer in range(2):
            for i in range(num_qubits - 1):
                circuit.cx(i, i + 1)

                for power in range(np.random.randint(4)):
                    circuit.s(i + 1)

                circuit.cx(i, i + 1)

            for i in range(num_qubits):
                circuit.s(i)
                circuit.sx(i)

                for power in range(np.random.randint(4)):
                    circuit.s(i)

                circuit.sx(i)
                circuit.s(i)

        circuit.measure_all()
        circuits.append(circuit)
        
    return circuits

### Zero Noise Extrapolation

In [19]:
def get_zne_circuits() -> list[qiskit.QuantumCircuit]:
    num_circuits = 9
    num_qubits = 21
    circuits = []
    
    for circuit_index in range(num_circuits):
        circuit = qiskit.QuantumCircuit(num_qubits)
        
        circuit.h(range(num_qubits))
        
        for repetition in range(circuit_index):
            circuit.h(range(num_qubits))
            circuit.h(range(num_qubits))
            
        for qubit in range(num_qubits - 1):
            circuit.cx(qubit, qubit + 1)
            
            for repetition in range(circuit_index):
                circuit.cx(qubit, qubit + 1)
                circuit.cx(qubit, qubit + 1)
                
        circuit.measure_all()
        circuits.append(circuit)
        
    return circuits

### Probabilistic Error Cancellation

In [20]:
def sample_and_append_operation(
    circuit: qiskit.QuantumCircuit,
    gate_choices: list[qiskit.circuit.singleton.SingletonGate],
    gate_probs: list[float],
    targets: list[int]
):
    sampled_operation = gate_choices[np.random.choice(len(gate_choices), p=gate_probs)]
    
    for op in sampled_operation:
        circuit.append(op, targets)
        
    return circuit
    

def get_pec_circuits() -> list[qiskit.QuantumCircuit]:
    num_circuits = 250
    num_layers = 8
    num_qubits = 8
    beta_values = [0.21, 0.24, 0.13, 0.1, 0.8, 0.1, 0.23, 0.5]
    gamma_values = [0.1, 0.2, 0.1, 0.13, 0.5, 0.6, 0.3, 0.5]
    sx_gate_probs = [0.97, 0.01, 0.01, 0.01]
    sx_gate_choices = [
        [qiskit.circuit.library.SXGate()],
        [qiskit.circuit.library.XGate()],
        [qiskit.circuit.library.ZGate()],
        [qiskit.circuit.library.XGate(), qiskit.circuit.library.XGate()],
    ]
    
    
    cnot_gate_probs = [0.92, 0.04, 0.04]
    cnot_gate_choices = [
        [qiskit.circuit.library.CXGate()],
        [qiskit.circuit.library.CZGate()],
        [qiskit.circuit.library.CYGate()],       
    ]
    
    
    circuits = []
    
    for circuit_index in range(num_circuits):
        circuit = qiskit.QuantumCircuit(num_qubits)
        
        circuit.h(range(num_qubits))
        
        for layer in range(num_layers):
            for qubit in range(num_qubits):
                circuit.s(qubit)
                sample_and_append_operation(circuit, sx_gate_choices, sx_gate_probs, [qubit])
                circuit.rz(beta_values[layer], qubit)
                sample_and_append_operation(circuit, sx_gate_choices, sx_gate_probs, [qubit])
                circuit.s(qubit)
                
            for qubit in range(num_qubits - 1):
                sample_and_append_operation(circuit, cnot_gate_choices, cnot_gate_probs, [qubit, qubit + 1])
                circuit.rz(gamma_values[layer], qubit + 1)
                sample_and_append_operation(circuit, cnot_gate_choices, cnot_gate_probs, [qubit, qubit + 1])
                
        circuit.measure_all()
        circuits.append(circuit)
        
    return circuits

### JigSaw

In [21]:
def get_jigsaw_circuits() -> list[qiskit.QuantumCircuit]:
    num_qubits = 13
    num_meas_qubits = 2
    secret_key = [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1]
    num_circuits = 32
    
    jigsaw_circuits = []
    measurement_indices = np.random.randint(0, 12, (num_circuits, 2))
    
    for circuit_index in range(num_circuits):
        circuit = qiskit.QuantumCircuit(num_qubits, 2)
        
        circuit.x(12)
        
        for qubit in range(13):
            circuit.h(qubit)
            
        for qubit in range(12):
            if secret_key[qubit]:
                circuit.cx(qubit, 12)
                
        for qubit in range(13):
            circuit.h(qubit)
            
        circuit.measure(measurement_indices[circuit_index][0], 0)
        circuit.measure(measurement_indices[circuit_index][1], 1)
        
        jigsaw_circuits.append(circuit)
            
    return jigsaw_circuits

### Randomized Compilation

In [22]:
def add_hard_cycle(
    circuit: qiskit.QuantumCircuit,
    cnot_pairs: list[tuple[int, int]],
    num_qubits: int,
) -> qiskit.QuantumCircuit:
    i = qiskit.circuit.library.IGate()
    x = qiskit.circuit.library.XGate()
    y = qiskit.circuit.library.YGate()
    z = qiskit.circuit.library.ZGate()
    
    pauli_left_frames = [
        [i, i], [i, x], [i, y], [i, z],
        [x, i], [x, x], [x, y], [x, z],
        [y, i], [y, x], [y, y], [y, z],
        [z, i], [z, x], [z, y], [z, z],
    ]
    
    pauli_right_frames = [
        [i, i], [i, x], [z, y], [z, z],
        [x, x], [x, i], [y, z], [y, y],
        [y, x], [y, i], [x, z], [x, y],
        [z, i], [z, x], [i, y], [i, z],
    ]

    pauli_frame_indices = np.random.randint(0, 16, (len(cnot_pairs), ))
    pauli_frame_indices_trivial = np.random.randint(0, 16, ((num_qubits + 1) // 2, ))
    flattened_cnot_qubits = [qubit for cnot_pair in cnot_pairs for qubit in cnot_pair]

    for qubit in range(num_qubits):
        if qubit not in flattened_cnot_qubits:
            circuit.append(pauli_left_frames[pauli_frame_indices_trivial[qubit // 2]][qubit % 2], [qubit])
    
    for pair_index, cnot_pair in enumerate(cnot_pairs):
        circuit.append(pauli_left_frames[pauli_frame_indices[pair_index]][0], [cnot_pair[0]])
        circuit.append(pauli_left_frames[pauli_frame_indices[pair_index]][1], [cnot_pair[1]])
        circuit.cx(cnot_pair[0], cnot_pair[1])
        circuit.append(pauli_right_frames[pauli_frame_indices[pair_index]][0],[cnot_pair[0]])
        circuit.append(pauli_right_frames[pauli_frame_indices[pair_index]][1], [cnot_pair[1]])
        
    for qubit in range(num_qubits):
        if qubit not in flattened_cnot_qubits:
            circuit.append(pauli_left_frames[pauli_frame_indices_trivial[qubit // 2]][qubit % 2], [qubit])
            
    return circuit

    
def get_rc_circuits() -> list[qiskit.QuantumCircuit]:
    num_qubits = 4
    num_circuits = 1024
    circuits = []
    
    for circuit_index in range(num_circuits):
        circuit = qiskit.QuantumCircuit(num_qubits)

        circuit.h(range(num_qubits))
  
        circuit.rz(np.pi / 4, 1)
        circuit.rz(np.pi / 8, 2)
        circuit.rz(np.pi / 8, 3)
        
        add_hard_cycle(circuit, [(1, 0)], num_qubits)
        
        circuit.rz(-np.pi / 4, 0)
        
        add_hard_cycle(circuit, [(1, 0)], num_qubits)
        
        circuit.rz(np.pi / 4, 0)
        
        add_hard_cycle(circuit, [(2, 0)], num_qubits)
        
        circuit.rz(-np.pi / 8, 0)
        
        add_hard_cycle(circuit, [(2, 0)], num_qubits)
        
        circuit.rz(np.pi / 8, 0)
        circuit.rz(np.pi / 4, 2)
        
        add_hard_cycle(circuit, [(3, 0), (2, 1)], num_qubits)
        
        circuit.rz(-np.pi / 16, 0)
        circuit.rz(-np.pi / 4, 1)
        
        add_hard_cycle(circuit, [(3, 0), (2, 1)], num_qubits)
        
        circuit.rz(np.pi / 16, 0)
        circuit.rz(np.pi / 4, 1)
        circuit.rz(np.pi / 8, 3)
        
        add_hard_cycle(circuit, [(3, 1)], num_qubits)
        
        circuit.rz(-np.pi / 8, 1)
        
        add_hard_cycle(circuit, [(3, 1)], num_qubits)
        
        circuit.rz(np.pi / 8, 1)
        circuit.rz(np.pi / 4, 3)
        
        add_hard_cycle(circuit, [(3, 2)], num_qubits)
        
        circuit.rz(-np.pi / 4, 2)
        
        add_hard_cycle(circuit, [(3, 2)], num_qubits)
        
        circuit.rz(np.pi / 4, 2)
        
        add_hard_cycle(circuit, [(3, 0), (2, 1)], num_qubits)
        add_hard_cycle(circuit, [(0, 3), (1, 2)], num_qubits)
        add_hard_cycle(circuit, [(3, 0), (2, 1)], num_qubits)
        
        circuit.measure_all()
        circuits.append(circuit)
        
    return circuits

### Cycle Benchmarking

In [23]:
def get_cycle_benchmarking_circuits() -> list[qiskit.QuantumCircuit]:
    num_qubits = 6
    pauli_cycle = [0, 1, 0, 1, 0, 1]
    num_pauli_randomizations = 64
    cycle_repetitions = [4, 8, 12]
    num_circuits_per_initial_state = 30
    num_circuits = 1920
    circuits = []
    
    x = qiskit.circuit.library.XGate()
    y = qiskit.circuit.library.YGate()
    z = qiskit.circuit.library.ZGate()
    
    pauli_gates = [x, y, z]
    edge_randomization_gates = [[y, y, x], [x, x, y], [x, y, z]]
    state_prep_gates = None
    
    for circuit_index in range(num_circuits):
        circuit_cycle_repetitions = cycle_repetitions[circuit_index % len(cycle_repetitions)]
        
        if not (circuit_index % num_circuits_per_initial_state):
            state_prep_gate_indices = np.random.randint(0, 3, (num_qubits,))
            
        frame_randomization_gate_indices = np.random.randint(
            0, 3, (circuit_cycle_repetitions + 1, num_qubits)
        )
        
        circuit = qiskit.QuantumCircuit(num_qubits)
        
        for qubit in range(num_qubits):
            circuit.append(
                edge_randomization_gates[state_prep_gate_indices[qubit]][
                    frame_randomization_gate_indices[0][qubit]
                ],
                [qubit],
            )
            
        for rep_index in range(1, circuit_cycle_repetitions):
            for qubit in range(num_qubits):
                circuit.append(pauli_gates[pauli_cycle[qubit]], [qubit])
                circuit.append(
                    pauli_gates[frame_randomization_gate_indices[rep_index][qubit]],
                    [qubit]
                )
                
        for qubit in range(num_qubits):
            circuit.append(pauli_gates[pauli_cycle[qubit]], [qubit])
            circuit.append(
                edge_randomization_gates[state_prep_gate_indices[qubit]][
                    frame_randomization_gate_indices[circuit_cycle_repetitions][qubit]
                ],
                [qubit],
            )
            
        circuit.measure_all()
        circuits.append(circuit)
        
    return circuits

### ACES

In [24]:
def add_dressed_aces_layer(
    circuit: qiskit.QuantumCircuit,
    cnot_indices: list[tuple[int, int]],
    hadamard_indices: list[int],
    s_indices: list[int],
) -> qiskit.QuantumCircuit:
    i = qiskit.circuit.library.IGate()
    x = qiskit.circuit.library.XGate()
    y = qiskit.circuit.library.YGate()
    z = qiskit.circuit.library.ZGate()

    cnot_pauli_left_frames = [
        [i, i], [i, x], [i, y], [i, z],
        [x, i], [x, x], [x, y], [x, z],
        [y, i], [y, x], [y, y], [y, z],
        [z, i], [z, x], [z, y], [z, z],
    ]
    
    cnot_pauli_right_frames = [
        [i, i], [i, x], [z, y], [z, z],
        [x, x], [x, i], [y, z], [y, y],
        [y, x], [y, i], [x, z], [x, y],
        [z, i], [z, x], [i, y], [i, z],
    ]
    
    hadamard_pauli_left_frames = [i, x, y, z]
    hadamard_pauli_right_frames = [i, y, x, z]
    s_pauli_left_frames = [i, x, y, z]
    s_pauli_right_frames = [i, z, y, x]
    
    cnot_pauli_indices = np.random.randint(0, 16, (len(cnot_indices),))
    hadamard_pauli_indices = np.random.randint(0, 4, (len(hadamard_indices),))
    s_pauli_indices = np.random.randint(0, 4, (len(s_indices),))
    
    for cnot_index, cnot_pair in enumerate(cnot_indices):
        control, target = cnot_pair

        circuit.append(cnot_pauli_left_frames[cnot_pauli_indices[cnot_index]][0], [control])
        circuit.append(cnot_pauli_left_frames[cnot_pauli_indices[cnot_index]][1], [target])
        
        circuit.cx(control, target)
        
        circuit.append(cnot_pauli_right_frames[cnot_pauli_indices[cnot_index]][0], [control])
        circuit.append(cnot_pauli_right_frames[cnot_pauli_indices[cnot_index]][1], [target])
        
    for qubit_index, qubit in enumerate(hadamard_indices):
        circuit.append(hadamard_pauli_left_frames[hadamard_pauli_indices[qubit_index]], [qubit])
        circuit.h(qubit)
        circuit.append(hadamard_pauli_right_frames[hadamard_pauli_indices[qubit_index]], [qubit])
        
    for qubit_index, qubit in enumerate(s_indices):
        circuit.append(s_pauli_left_frames[s_pauli_indices[qubit_index]], [qubit])
        circuit.s(qubit)
        circuit.append(s_pauli_right_frames[s_pauli_indices[qubit_index]], [qubit])
        
    return circuit
    

def get_aces_circuits(backend: IBMBackend) -> list[qiskit.QuantumCircuit]:
    num_qubits = 5
    num_base_circuits = 20
    num_randomizations_per_circuit = 10
    min_depth = 2
    max_depth = 10
    num_circuits = num_base_circuits * num_randomizations_per_circuit
    circuits = []
    
    connectivity = collections.defaultdict(list)
    
    for edge in backend.coupling_map:
        connectivity[edge[0]].append(edge[1])
        connectivity[edge[1]].append(edge[0])
    
    depth = 0
    cnot_indices = []
    hadamard_indices = []
    s_indices = []
    
    for circuit_index in range(num_circuits):
        circuit = qiskit.QuantumCircuit(num_qubits)
        
        if not (circuit_index % num_randomizations_per_circuit):
            depth = np.random.randint(min_depth, max_depth + 1)
            cnot_indices = []
            hadamard_indices = []
            s_indices = []
        
            for layer in range(depth):
                max_num_cnots = np.random.randint(0, num_qubits // 2 + 1)
                cnot_indices.append(get_cnot_layer_indices(connectivity, range(num_qubits), max_num_cnots))

                num_hadamards = np.random.randint(0, num_qubits - (2 * len(cnot_indices[-1])) + 1)
                hadamard_probs = np.ones((num_qubits,))
                hadamard_probs[[qubit for cnot_pair in cnot_indices[-1] for qubit in cnot_pair]] = 0.0
                hadamard_probs /= np.sum(hadamard_probs)
                hadamard_indices.append(list(np.random.choice(num_qubits, num_hadamards, p=hadamard_probs)))

                hadamard_probs[hadamard_indices[-1]] = 0
                s_indices.append(list(np.argwhere(hadamard_probs).flatten()))

        for layer in range(depth):
            add_dressed_aces_layer(circuit, cnot_indices[layer], hadamard_indices[layer], s_indices[layer])
            
        circuit.measure_all()
        circuits.append(circuit)
        
    return circuits

### RCS with mirror circuits

In [53]:
def get_rcs_mirror_circuits(backend: IBMBackend) -> list[qiskit.QuantumCircuit]:
    num_circuits = 640
    depths = [2 * i for i in range(1, 21)]
    num_qubits = 5
    num_circuits_per_depth = 32
    circuits = []
 
    i = qiskit.circuit.library.IGate()
    x = qiskit.circuit.library.XGate()
    y = qiskit.circuit.library.YGate()
    z = qiskit.circuit.library.ZGate()
    h = qiskit.circuit.library.HGate()
    
    pauli_gates = [i, x, y, z]
    clifford_gates = [x, y, z, h]
    
    connectivity = collections.defaultdict(list)
    
    for edge in backend.coupling_map:
        connectivity[edge[0]].append(edge[1])
        connectivity[edge[1]].append(edge[0])
    
    for circuit_index in range(num_circuits):
        circuit_depth = depths[circuit_index // num_circuits_per_depth]
        num_cnots = np.random.randint(0, (num_qubits // 2) + 1, (circuit_depth // 2,))
        pauli_indices = np.random.randint(0, 4, (circuit_depth + 1, num_qubits))
        clifford_indices = np.random.randint(0, 4, (circuit_depth // 2, num_qubits))
        initial_clifford_indices = np.random.randint(0, 4, (num_qubits,))
        
        circuit = qiskit.QuantumCircuit(num_qubits)
        
        cnot_layer_indices = [
            get_cnot_layer_indices(
                connectivity,
                range(num_qubits),
                num_cnots_in_layer,
            ).flatten()
            for num_cnots_in_layer in num_cnots
        ]

        for qubit in range(num_qubits):
            circuit.append(clifford_gates[initial_clifford_indices[qubit]], [qubit])
            circuit.append(pauli_gates[pauli_indices[0, qubit]], [qubit])
        
        for layer in range(circuit_depth // 2):
            for cnot_index in range(cnot_layer_indices[layer].shape[0] // 2):
                circuit.cx(
                    cnot_layer_indices[layer][2 * cnot_index],
                    cnot_layer_indices[layer][2 * cnot_index + 1]
                )
                
            for qubit in range(num_qubits):
                if qubit not in cnot_layer_indices[layer]:
                    circuit.append(clifford_gates[clifford_indices[layer, qubit]], [qubit])
                
                circuit.append(pauli_gates[pauli_indices[layer + 1, qubit]], [qubit])
            
        for inv_layer in range(circuit_depth // 2 - 1, -1, -1):
            for cnot_index in range(cnot_layer_indices[inv_layer].shape[0] // 2):
                circuit.cx(
                    cnot_layer_indices[inv_layer][2 * cnot_index],
                    cnot_layer_indices[inv_layer][2 * cnot_index + 1]
                )
                
            for qubit in range(num_qubits):
                if qubit not in cnot_layer_indices[inv_layer]:
                    circuit.append(clifford_gates[clifford_indices[inv_layer, qubit]], [qubit])
            
                circuit.append(pauli_gates[pauli_indices[circuit_depth - inv_layer, qubit]], [qubit])
            
        for qubit in range(num_qubits):
            circuit.append(clifford_gates[initial_clifford_indices[qubit]], [qubit])
            
        circuit.measure_all()
        circuits.append(circuit)
        
    return circuits

### Quantum Volume

In [76]:
def get_quantum_volume_circuits(backend: IBMBackend) -> list[qiskit.QuantumCircuit]:
    circuits = []
    min_qubits = 2
    max_qubits = 6
    circuits_per_size = 100
    
    for num_qubits in range(min_qubits, max_qubits + 1):
        for circuit_index in range(circuits_per_size):
            circuits.append(
                qiskit.circuit.library.QuantumVolume(
                    num_qubits,
                    seed=(num_qubits * circuits_per_size) + circuit_index,
                    flatten=True,
                ).decompose(),
            )
            
    return circuits

### CLOPS_h

In [96]:
def get_clops_circuits(backend: IBMBackend) -> list[qiskit.QuantumCircuit]:
    num_circuits = 100
    num_parameterizations_per_circuit = 10
    num_qubits = backend.num_qubits
    num_layers = num_qubits
    
    connectivity = collections.defaultdict(list)
    
    for edge in backend.coupling_map:
        connectivity[edge[0]].append(edge[1])
        connectivity[edge[1]].append(edge[0])
    
    circuits = []

    for circuit_index in range(num_circuits // num_parameterizations_per_circuit):
        ecr_indices = [
            get_cnot_layer_indices(connectivity, range(num_qubits), num_qubits // 2).flatten()
            for i in range(num_layers)
        ]
        
        for parameter_index in range(num_parameterizations_per_circuit):
            circuit = qiskit.QuantumCircuit(num_qubits)
            rz_angles = 2 * np.pi * np.random.sample((num_layers, num_qubits, ))
            
            for layer in range(num_layers):
                for cnot_index in range(len(ecr_indices[layer]) // 2):
                    circuit.ecr(
                        ecr_indices[layer][2 * cnot_index],
                        ecr_indices[layer][2 * cnot_index + 1]
                    )
                    
                for qubit in range(num_qubits):
                    circuit.sx(qubit)
                    circuit.x(qubit)
                    circuit.rz(rz_angles[layer, qubit], qubit)
                    
            circuits.append(circuit)
            
    return circuits

### Characterizing topological order

In [ ]:
def get_characterizing_topological_order_circuits(backend: IBMBackend) -> list[qiskit.QuantumCircuit]:
    pass

### Fidelity estimation

### Hamiltonian energy estimation

### Quantum process tomography

### Entanglement entropy prediction

### Gate cutting

### Wire cutting

### Robust shadow estimation

### Energy estimation + ZNE + RC

### ZNE + CDR

In [3]:
backend_name = "ibm_kyiv"
backend = QiskitRuntimeService(
    channel="ibm_quantum",
    token="c0ed0f977fc96d511605341385f656a6f645fad36841935e8607da73c66b88d94fc4f0039f31fcb5deb38050ed690e95d6bdef43a2601351aff0d547a0da5af8"
).backend(backend_name)

cdr_circuits = get_cdr_circuits()
compiled_circuits = qiskit.compiler.transpile(cdr_circuits, backend, optimization_level=1)
openqasm_circuits = [qiskit.qasm3.dumps(circuit) for circuit in compiled_circuits]


KeyboardInterrupt



In [63]:
compiled_circuits[0].count_ops(), cdr_circuits[0].count_ops()

(OrderedDict([('rz', 1029),
              ('sx', 449),
              ('ecr', 129),
              ('x', 60),
              ('measure', 16),
              ('barrier', 1)]),
 OrderedDict([('s', 148),
              ('sx', 64),
              ('cx', 60),
              ('h', 16),
              ('measure', 16),
              ('barrier', 1)]))

In [65]:
compiled_circuits[0].count_ops(), cdr_circuits[0].count_ops()

(OrderedDict([('rz', 98),
              ('sx', 68),
              ('ecr', 40),
              ('x', 20),
              ('measure', 16),
              ('barrier', 1)]),
 OrderedDict([('s', 150),
              ('sx', 64),
              ('cx', 60),
              ('h', 16),
              ('measure', 16),
              ('barrier', 1)]))

In [60]:
openqasm_circuits[0]

'OPENQASM 3.0;\ninclude "stdgates.inc";\ngate rzx(p0) _gate_q_0, _gate_q_1 {\n  h _gate_q_1;\n  cx _gate_q_0, _gate_q_1;\n  rz(p0) _gate_q_1;\n  cx _gate_q_0, _gate_q_1;\n  h _gate_q_1;\n}\ngate ecr _gate_q_0, _gate_q_1 {\n  rzx(pi/4) _gate_q_0, _gate_q_1;\n  x _gate_q_0;\n  rzx(-pi/4) _gate_q_0, _gate_q_1;\n}\nbit[16] meas;\nx $53;\nrz(pi/2) $53;\nx $60;\nrz(pi/2) $60;\nx $61;\nrz(pi/2) $61;\nsx $62;\nrz(pi/2) $62;\nrz(pi/2) $72;\nsx $72;\necr $72, $62;\nx $72;\nrz(-pi/2) $72;\nrz(-pi/2) $62;\nsx $62;\nrz(-pi) $62;\necr $72, $62;\nsx $62;\nrz(-pi/2) $62;\necr $61, $62;\nrz(pi/2) $61;\nsx $62;\necr $61, $62;\nrz(-pi) $61;\nsx $61;\necr $60, $61;\nrz(pi/2) $60;\nsx $61;\necr $60, $61;\nrz(-pi) $60;\nsx $60;\necr $53, $60;\nsx $53;\nrz(pi/2) $53;\nsx $60;\necr $53, $60;\nrz(pi/2) $53;\nsx $53;\nrz(pi/2) $53;\nx $60;\nrz(pi/2) $60;\nrz(-pi) $61;\nx $61;\nrz(pi/2) $62;\nsx $62;\necr $61, $62;\nsx $61;\nrz(pi/2) $61;\nsx $62;\necr $61, $62;\nrz(-pi) $61;\nsx $61;\necr $60, $61;\nrz(pi/2) $6